In [ ]:
import glob
import os
import pandas as pd
import numpy as np
import h5py
import xarray as xr
from datetime import datetime, timedelta
from shapely.geometry import Point
import geopandas as gpd
import dask.array as da

directory = input('Input file directory')
drop_list = ['Profile_UTC_Time', 'Day_Night_Flag', 'Off_Nadir_Angle', 'Solar_Zenith_Angle', 'Solar_Azimuth_Angle', 'Scattering_Angle', 'Spacecraft_Position', 'Parallel_Column_Reflectance_532', 'Parallel_Column_Reflectance_Uncertainty_532', 'Parallel_Column_Reflectance_RMS_Variation_532', 'Perpendicular_Column_Reflectance_532', 'Perpendicular_Column_Reflectance_Uncertainty_532', 'Perpendicular_Column_Reflectance_RMS_Variation_532', 'Column_Integrated_Attenuated_Backscatter_532', 'Column_IAB_Cumulative_Probability', 'Column_Optical_Depth_Cloud_532', 'Column_Optical_Depth_Cloud_Uncertainty_532', 'Column_Optical_Depth_Tropospheric_Aerosols_532', 'Column_Optical_Depth_Tropospheric_Aerosols_Uncertainty_532', 'Column_Optical_Depth_Stratospheric_Aerosols_532', 'Column_Optical_Depth_Stratospheric_Aerosols_Uncertainty_532', 'Column_Optical_Depth_Tropospheric_Aerosols_1064', 'Column_Optical_Depth_Tropospheric_Aerosols_Uncertainty_1064', 'Column_Optical_Depth_Stratospheric_Aerosols_1064', 'Column_Optical_Depth_Stratospheric_Aerosols_Uncertainty_1064', 'Column_Feature_Fraction', 'Tropopause_Height', 'Tropopause_Temperature', 'IGBP_Surface_Type', 'Snow_Ice_Surface_Type', 'DEM_Surface_Elevation', 'Minimum_Laser_Energy_532', 'Normalization_Constant_Uncertainty', 'Calibration_Altitude_532', 'FeatureFinderQC', 'High_Resolution_Layers_Cleared', 'Number_Layers_Found', 'Surface_Wind_Speed', 'Layer_Base_Extended', 'Layer_Top_Pressure', 'Midlayer_Pressure', 'Layer_Base_Pressure', 'Layer_Top_Temperature', 'Layer_Centroid_Temperature', 'Midlayer_Temperature', 'Layer_Base_Temperature', 'Opacity_Flag', 'Horizontal_Averaging', 'Attenuated_Scattering_Ratio_Statistics_532', 'Attenuated_Backscatter_Statistics_532', 'Integrated_Attenuated_Backscatter_532', 'Integrated_Attenuated_Backscatter_Uncertainty_532', 'Attenuated_Backscatter_Statistics_1064', 'Integrated_Attenuated_Backscatter_1064', 'Integrated_Attenuated_Backscatter_Uncertainty_1064', 'Volume_Depolarization_Ratio_Statistics', 'Integrated_Volume_Depolarization_Ratio', 'Integrated_Volume_Depolarization_Ratio_Uncertainty', 'Attenuated_Total_Color_Ratio_Statistics', 'Integrated_Attenuated_Total_Color_Ratio', 'Integrated_Attenuated_Total_Color_Ratio_Uncertainty', 'Overlying_Integrated_Attenuated_Backscatter_532', 'Layer_IAB_QA_Factor', 'Feature_Classification_Flags', 'ExtinctionQC_532', 'ExtinctionQC_1064', 'CAD_Score', 'Initial_CAD_Score', 'Measured_Two_Way_Transmittance_532', 'Measured_Two_Way_Transmittance_Uncertainty_532', 'Two_Way_Transmittance_Measurement_Region', 'Feature_Optical_Depth_532', 'Feature_Optical_Depth_Uncertainty_532', 'Initial_532_Lidar_Ratio', 'Final_532_Lidar_Ratio', 'Final_532_Lidar_Ratio_Uncertainty', 'Lidar_Ratio_532_Selection_Method', 'Layer_Effective_532_Multiple_Scattering_Factor', 'Integrated_Particulate_Depolarization_Ratio', 'Integrated_Particulate_Depolarization_Ratio_Uncertainty', 'Particulate_Depolarization_Ratio_Statistics', 'Feature_Optical_Depth_1064', 'Feature_Optical_Depth_Uncertainty_1064', 'Initial_1064_Lidar_Ratio', 'Final_1064_Lidar_Ratio', 'Final_1064_Lidar_Ratio_Uncertainty', 'Lidar_Ratio_1064_Selection_Method', 'Layer_Effective_1064_Multiple_Scattering_Factor', 'Integrated_Particulate_Color_Ratio', 'Integrated_Particulate_Color_Ratio_Uncertainty', 'Particulate_Color_Ratio_Statistics', 'Relative_Humidity', 'Single_Shot_Cloud_Cleared_Fraction', 'Ice_Water_Path', 'Ice_Water_Path_Uncertainty', 'Surface_Top_Altitude_532', 'Surface_Base_Altitude_532', 'Surface_Integrated_Attenuated_Backscatter_532', 'Surface_532_Integrated_Depolarization_Ratio', 'Surface_532_Integrated_Attenuated_Color_Ratio', 'Surface_Detection_Flags_532', 'Surface_Detection_Confidence_532', 'Surface_Overlying_Integrated_Attenuated_Backscatter_532', 'Surface_Scaled_RMS_Background_532', 'Surface_Peak_Signal_532', 'Surface_Detections_333m_532', 'Surface_Detections_1km_532', 'Surface_Top_Altitude_1064', 'Surface_Base_Altitude_1064', 'Surface_Integrated_Attenuated_Backscatter_1064', 'Surface_1064_Integrated_Depolarization_Ratio', 'Surface_1064_Integrated_Attenuated_Color_Ratio', 'Surface_Detection_Flags_1064', 'Surface_Detection_Confidence_1064', 'Surface_Overlying_Integrated_Attenuated_Backscatter_1064', 'Surface_Scaled_RMS_Background_1064', 'Surface_Peak_Signal_1064', 'Surface_Detections_333m_1064', 'Surface_Detections_1km_1064', 'ssProfile_ID', 'ssLatitude', 'ssLongitude', 'ssProfile_Time', 'ssProfile_UTC_Time', 'ssParallel_Column_Reflectance_532', 'ssParallel_Column_Reflectance_Uncertainty_532', 'ssPerpendicular_Column_Reflectance_532', 'ssPerpendicular_Column_Reflectance_Uncertainty_532', 'ssColumn_Integrated_Attenuated_Backscatter_532', 'ssColumn_IAB_Cumulative_Probability', 'ssDEM_Surface_Elevation', 'ssLaser_Energy_532', 'ssNumber_Layers_Found', 'ssLayer_Top_Altitude', 'ssLayer_Base_Altitude', 'ssLayer_Top_Pressure', 'ssMidlayer_Pressure', 'ssLayer_Base_Pressure', 'ssLayer_Top_Temperature', 'ssLayer_Centroid_Temperature', 'ssMidlayer_Temperature', 'ssLayer_Base_Temperature', 'ssOpacity_Flag', 'ssAttenuated_Scattering_Ratio_Statistics_532', 'ssAttenuated_Backscatter_Statistics_532', 'ssIntegrated_Attenuated_Backscatter_532', 'ssIntegrated_Attenuated_Backscatter_Uncertainty_532', 'ssAttenuated_Backscatter_Statistics_1064', 'ssIntegrated_Attenuated_Backscatter_1064', 'ssIntegrated_Attenuated_Backscatter_Uncertainty_1064', 'ssVolume_Depolarization_Ratio_Statistics', 'ssIntegrated_Volume_Depolarization_Ratio', 'ssIntegrated_Volume_Depolarization_Ratio_Uncertainty', 'ssAttenuated_Total_Color_Ratio_Statistics', 'ssIntegrated_Attenuated_Total_Color_Ratio', 'ssIntegrated_Attenuated_Total_Color_Ratio_Uncertainty', 'ssOverlying_Integrated_Attenuated_Backscatter_532', 'ssLayer_IAB_QA_Factor', 'ssCAD_Score', 'ssInitial_CAD_Score', 'ssFeature_Classification_Flags', 'ssWas_Cleared', 'ssSurface_Top_Altitude_532', 'ssSurface_Base_Altitude_532', 'ssSurface_Integrated_Attenuated_Backscatter_532', 'ssSurface_532_Integrated_Depolarization_Ratio', 'ssSurface_532_Integrated_Attenuated_Color_Ratio', 'ssSurface_Detection_Flags_532', 'ssSurface_Detection_Confidence_532', 'ssSurface_Overlying_Integrated_Attenuated_Backscatter_532', 'ssSurface_Scaled_RMS_Background_532', 'ssSurface_Peak_Signal_532', 'ssSurface_Top_Altitude_1064', 'ssSurface_Base_Altitude_1064', 'ssSurface_Integrated_Attenuated_Backscatter_1064', 'ssSurface_1064_Integrated_Depolarization_Ratio', 'ssSurface_1064_Integrated_Attenuated_Color_Ratio', 'ssSurface_Detection_Flags_1064', 'ssSurface_Detection_Confidence_1064', 'ssSurface_Overlying_Integrated_Attenuated_Backscatter_1064', 'ssSurface_Scaled_RMS_Background_1064', 'ssSurface_Peak_Signal_1064']

# Get CALIPSO Data

In [ ]:

def days_to_date(input_str):
    try:
        # Split the input string to separate days and time
        days, time = str(input_str).split('days')

        # Convert the days part to a timedelta object
        duration = timedelta(days=float(days))

        # Define the reference date (1993, 1, 1)
        reference_date = datetime(1993, 1, 1)

        # Calculate the end date by adding the duration to the reference date
        end_date = reference_date + duration

        # Convert the time part to a timedelta object
        hours, minutes, seconds = map(float, time.split(':'))
        time_delta = timedelta(hours=hours, minutes=minutes, seconds=seconds)

        # Add the time delta to the end date
        end_date_with_time = end_date + time_delta

        return end_date_with_time
    except Exception as e:
       return 0  


def XRenderCALIPSO(filename):
        
        dataset = xr.open_dataset(filename, engine="netcdf4")
        params = np.array(['Longitude','Latitude','Profile_Time','Profile_ID'])
        data = []
        for param in params:
            param_data = np.array(dataset[param]).flatten()
            param_df = pd.DataFrame(param_data, columns=[param])
            data.append(param_df)
        attributes = ['Layer_Top_Altitude','Layer_Base_Altitude','Layer_Base_Altitude']
        for attribute in attributes:
            param_data = pd.DataFrame(np.array(dataset[attribute]))
            param_df = pd.DataFrame(param_data.apply(lambda row: ','.join(row.dropna().astype(str)), axis=1), columns=[attribute])
            data.append(param_df[attribute])
        data = pd.concat(data, axis=1)
        data['Time0'] = data['Profile_Time'].apply(days_to_date)
        
        data['filename'] = pd.DataFrame([os.path.basename(filename)] * len(data), columns=['filename'])
        data['Lat0'] = data['Latitude']
        data['Long0'] = data['Longitude']
        data = data.drop(columns = ['Profile_Time','Longitude','Latitude'])
        data = data.drop(data[data['Profile_ID'] == ''].index)
        return data

# Get ATL09 Data


In [ ]:
def gps_to_datetime(gps_seconds):
    gps_epoch = datetime(2018, 1, 1)
    gps_time = gps_epoch + timedelta(seconds=gps_seconds)
    return gps_time


def HDFtoDF(filename, chunk_size=408):
    try:
        # Open the HDF file using h5py
        with h5py.File(filename, 'r') as file:
            # Access latitude, longitude, and time data using Dask
            latitude_data = da.from_array(file['profile_1']['high_rate']['latitude'],chunks = chunk_size)
            longitude_data = da.from_array(file['profile_1']['high_rate']['longitude'], chunks=chunk_size)
            delta_time_data = da.from_array(file['profile_1']['high_rate']['delta_time'], chunks=chunk_size)
            
            # Convert Dask arrays to Pandas DataFrame
            df = pd.DataFrame({
                'Lat0': latitude_data.compute(),
                'Long0': longitude_data.compute(),
                'Time0': delta_time_data.compute()
            })
            df['Time0'] = df['Time0'].apply(gps_to_datetime)
            filename_df = pd.DataFrame([os.path.basename(filename)] * len(df), columns=['filename'])
            df = pd.concat([filename_df,df],axis = 1)
            attributes = ['profile_1/high_rate/layer_top','profile_1/high_rate/layer_bot','profile_1/high_rate/ds_va_bin_h']
            dataset = []
            for attribute in attributes:
                param_data = pd.DataFrame(np.array(file[attribute]))
                param_df = pd.DataFrame(param_data.apply(lambda row: ','.join(row.dropna().astype(str)), axis=1), columns=[attribute])
                dataset.append(param_df)
            dataset = pd.concat(dataset,axis = 1)
            df = pd.concat([df,dataset],axis = 1)
            print(f"{filename} successfully processed!")
        return df

    except Exception as e:
        print(f'Error reading {filename} due to {e}')



# Get Collocations

In [ ]:
def collocations_main(atl_data, modis_data, time_threshold_hours=1):

    threshold_distance = 0.01

    # Create GeoDataFrame for ATL data
    geometry_atl = [Point(xy) for xy in zip(atl_data['Long0'], atl_data['Lat0'])]
    gdf_atl = gpd.GeoDataFrame(atl_data, geometry=geometry_atl)

    # Create GeoDataFrame for MODIS data
    geometry_modis = [Point(xy) for xy in zip(modis_data['Long0'], modis_data['Lat0'])]
    gdf_modis = gpd.GeoDataFrame(modis_data, geometry=geometry_modis)

    # Perform a spatial join based on proximity with a specified threshold distance
    result = gdf_atl.sjoin_nearest(gdf_modis, how='inner',max_distance = threshold_distance)

    result['Time0_left'] = pd.to_datetime(result['Time0_left'])
    result['Time0_right'] = pd.to_datetime(result['Time0_right'])

    # Adjust the filtering based on the actual column names in the result
    if 'Time0_left' in result.columns and 'Time0_right' in result.columns:
        # Filter based on time values if time_threshold_hours is provided
        if time_threshold_hours is not None:
            result = result[
                (result['Time0_left'] == result['Time0_right']) |  # Changed to 'or' to handle time threshold
                (abs((result['Time0_left'] - result['Time0_right']).dt.total_seconds()) / 3600 <= time_threshold_hours) |  # Changed to 'or'
                (result['Time0_left'] == result['Time0_right'] + pd.Timedelta(hours=time_threshold_hours)) |  # Adding threshold
                (result['Time0_left'] == result['Time0_right'] - pd.Timedelta(hours=time_threshold_hours))  # Subtracting threshold

            ]
        else:
            # Filter only based on spatial threshold if time_threshold_hours is None
            result = result[result.geometry.distance(result.geometry) <= threshold_distance]
    else:
        print("Time columns not found in the result.")

    return result

In [ ]:
CALIPSOFiles = sorted(glob.glob(os.path.join(directory, '**/*.hdf'), recursive=True))
atlfiles = sorted(glob.glob(os.path.join(directory, '**/*.h5',), recursive=True))
collocations = []
for i in range(len(atlfiles)):
    try:
        calipso = XRenderCALIPSO(CALIPSOFiles[i])
        atl = HDFtoDF(atlfiles[i])
        collocations.append(collocations_main(atl,calipso))
    except:
        continue
collocations = pd.concat(collocations)
collocations.to_csv(f'{directory}/collocationswhole.csv')